In [ ]:
%pip install PyTDC

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.7/107.7 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 20.3 MB/s eta 0:00:00
  Created wheel for PyTDC: filename=PyTDC-0.4.1-py3-none-any.whl size=140644 sha256=9fd68c14a6c8c029a178db87a69626115daf2337cfd1eb6e94e080ad36eae3ce
  Stored in directory: /root/.cache/pip/wheels/14/b7/b8/9d9e1442129743359b0507234ccc0beb67b47173faccd57d54
Successfully built PyTDC


In [ ]:
from tdc.single_pred import ADME
from tdc.benchmark_group import admet_group
from tdc import BenchmarkGroup
import pandas as pd
import numpy as np
from sklearn import metrics

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm #progress bar
import rdkit
from rdkit import Chem #Chemistry
from rdkit.Chem import rdMolDescriptors #molecular descriptors
from rdkit.Chem import PandasTools
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
rdkit.__version__

import xgboost
import sklearn
import tensorflow as tf
from xgboost import XGBClassifier
from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import fbeta_score, make_scorer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten
from sklearn.model_selection import train_test_split #ML training
from sklearn.model_selection import RandomizedSearchCV
#from yellowbrick.regressor import prediction_error, ResidualsPlot
from tdc.single_pred import ADME
from tdc.benchmark_group import admet_group
from tdc import BenchmarkGroup
from tdc import Evaluator
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def pred_prob_to_score(pred_prob):
    res = []
    for i in range(len(pred_prob)):
        res.append(pred_prob[i][1])
    res = np.array(res)
    return res

group = admet_group(path = 'data/')
name = 'DILI' #you need to change dataset name to get the model fusion result
benchmark = group.get(name)
train_val, test = benchmark['train_val'], benchmark['test']
y_test = np.array(test.Y)
for seed in [1,2,3,4,5]:
  train, valid = group.get_train_valid_split(benchmark = name, split_type = 'default', seed = seed)
  globals()['y_train_%s'%seed] = train.Y
  globals()['y_valid_%s'%seed] = valid.Y

predictions_val_xgb = pd.read_csv('%s_predictions_val_xgb.csv' % name)
predictions_val_rf = pd.read_csv('%s_predictions_val_rf.csv' % name)
predictions_val_svm = pd.read_csv('%s_predictions_val_svm.csv' % name)
predictions_val_adb = pd.read_csv('%s_predictions_val_adb.csv' % name)
predictions_val_cnn = pd.read_csv('%s_predictions_val_cnn.csv' % name)

predictions_test_xgb = pd.read_csv('%s_predictions_test_xgb.csv' % name)
predictions_test_rf = pd.read_csv('%s_predictions_test_rf.csv' % name)
predictions_test_svm = pd.read_csv('%s_predictions_test_svm.csv' % name)
predictions_test_adb = pd.read_csv('%s_predictions_test_adb.csv' % name)
predictions_test_cnn = pd.read_csv('%s_predictions_test_cnn.csv' % name)

def score_to_rank(array):
  res = np.argsort(np.flip(np.argsort(array)))+1
  return res

def normalize(array): # define function for normalization of scores
  maximum = np.max(array)
  minimum = np.min(array)
  norm_list = []
  for i in range(len(array)):
      norm_list.append((array[i]-minimum)/(maximum-minimum))
  return np.array(norm_list)

def df_to_list(df):
  res = []
  for col in df.columns:
    res.append(df[col].to_numpy())
  return res

predictions_val_xgb = [np.array(predictions_val_xgb.iloc[:,i][~np.isnan(predictions_val_xgb.iloc[:,i])]) for i in range(predictions_val_xgb.shape[1])]
predictions_val_rf = [np.array(predictions_val_rf.iloc[:,i][~np.isnan(predictions_val_rf.iloc[:,i])]) for i in range(predictions_val_rf.shape[1])]
predictions_val_svm = [np.array(predictions_val_svm.iloc[:,i][~np.isnan(predictions_val_svm.iloc[:,i])]) for i in range(predictions_val_svm.shape[1])]
predictions_val_adb = [np.array(predictions_val_adb.iloc[:,i][~np.isnan(predictions_val_adb.iloc[:,i])]) for i in range(predictions_val_adb.shape[1])]
predictions_val_cnn = [np.array(predictions_val_cnn.iloc[:,i][~np.isnan(predictions_val_cnn.iloc[:,i])]) for i in range(predictions_val_cnn.shape[1])]

predictions_test_xgb = [np.array(predictions_test_xgb.iloc[:,i][~np.isnan(predictions_test_xgb.iloc[:,i])]) for i in range(predictions_test_xgb.shape[1])]
predictions_test_rf = [np.array(predictions_test_rf.iloc[:,i][~np.isnan(predictions_test_rf.iloc[:,i])]) for i in range(predictions_test_rf.shape[1])]
predictions_test_svm = [np.array(predictions_test_svm.iloc[:,i][~np.isnan(predictions_test_svm.iloc[:,i])]) for i in range(predictions_test_svm.shape[1])]
predictions_test_adb = [np.array(predictions_test_adb.iloc[:,i][~np.isnan(predictions_test_adb.iloc[:,i])]) for i in range(predictions_test_adb.shape[1])]
predictions_test_cnn = [np.array(predictions_test_cnn.iloc[:,i][~np.isnan(predictions_test_cnn.iloc[:,i])]) for i in range(predictions_test_cnn.shape[1])]
scoreSys = ['xgb', 'rf', 'svm', 'adb', 'cnn']

ds_score = [[] for _ in range(5)]
for sys in scoreSys:
  for seed in range(len(ds_score)):
    loc = scoreSys.index(sys)
    scoreSys.remove(sys)
    ds = 0
    for i in range(len(scoreSys)-1):
      ds += np.sum(np.square(normalize(np.sort(globals()['predictions_val_%s' % sys][seed]))-normalize(np.sort(globals()['predictions_val_%s' % scoreSys[i]][seed]))))
    ds = ds/len(scoreSys)
    scoreSys.insert(loc, sys)
    ds_score[seed].append(ds)

ds_rank = np.reciprocal(ds_score)

def get_auprc(y_pred_proba, y_true):
  evaluator = Evaluator(name = 'PR-AUC')
  res = evaluator(y_true, y_pred_proba)
  return res

def get_auroc(y_pred_proba, y_true):
  evaluator = Evaluator(name = 'ROC-AUC')
  res = evaluator(y_true, y_pred_proba)
  #res = metrics.roc_auc_score(y_true, y_pred_proba)
  return res

def get_spearman_corr(y_pred_rank, y_true_rank):
  evaluator = Evaluator(name = 'Spearman')
  res = evaluator(y_true_rank, y_pred_rank)
  return res

def get_mae(y_pred, y_true):
  evaluator = Evaluator(name = 'MAE')
  res = evaluator(y_true,y_pred)
  return res

ps_score = [[] for _ in range(5)]

mae_datasets = ['Caco2_Wang','Lipophilicity_AstraZeneca','Solubility_AqSolDB','PPBR_AZ','LD50_Zhu']
spearman_datasets = ['VDss_Lombardo','Half_Life_Obach','Clearance_Hepatocyte_AZ','Clearance_Microsome_AZ']
auroc_datasets = ['Bioavailability_Ma','HIA_Hou','Pgp_Broccatelli','BBB_Martins','CYP3A4_Substrate_CarbonMangels','hERG','AMES','DILI']
auprc_datasets = ['CYP2C9_Veith','CYP2D6_Veith','CYP3A4_Veith','CYP2C9_Substrate_CarbonMangels','CYP2D6_Substrate_CarbonMangels']

if name in mae_datasets:
  for sys in scoreSys:
    for seed in range(len(ps_score)):
      n = len(globals()['predictions_val_%s' % sys][seed][~np.isnan(globals()['predictions_val_%s' % sys][seed])])
      m = len(globals()['y_valid_%s' % (seed+1)])
      if n >= m:
        ps = get_mae(globals()['predictions_val_%s' % sys][seed][:m], globals()['y_valid_%s' % (seed+1)])
      else:
        ps = get_mae(globals()['predictions_val_%s' % sys][seed][~np.isnan(globals()['predictions_val_%s' % sys][seed])], globals()['y_valid_%s' % (seed+1)][:n])
      ps_score[seed].append(ps)
elif name in spearman_datasets:
  for sys in scoreSys:
    for seed in range(len(ps_score)):
      n = len(globals()['predictions_val_%s' % sys][seed][~np.isnan(globals()['predictions_val_%s' % sys][seed])])
      m = len(globals()['y_valid_%s' % (seed+1)])
      if n >= m:
        ps = get_spearman_corr(score_to_rank(globals()['predictions_val_%s' % sys][seed][:m]), score_to_rank(globals()['y_valid_%s' % (seed+1)]))
      else:
        ps = get_spearman_corr(score_to_rank(globals()['predictions_val_%s' % sys][seed][~np.isnan(globals()['predictions_val_%s' % sys][seed])]), score_to_rank(globals()['y_valid_%s' % (seed+1)][:n]))
      ps_score[seed].append(ps)
elif name in auroc_datasets:
  for sys in scoreSys:
    for seed in range(len(ps_score)):
      n = len(globals()['predictions_val_%s' % sys][seed][~np.isnan(globals()['predictions_val_%s' % sys][seed])])
      m = len(globals()['y_valid_%s' % (seed+1)])
      if n >= m:
        ps = get_auroc(globals()['predictions_val_%s' % sys][seed][:m], globals()['y_valid_%s' % (seed+1)])
      else:
        ps = get_auroc(globals()['predictions_val_%s' % sys][seed][~np.isnan(globals()['predictions_val_%s' % sys][seed])], globals()['y_valid_%s' % (seed+1)][:n])
      ps_score[seed].append(ps)
elif name in auprc_datasets:
  for sys in scoreSys:
    for seed in range(len(ps_score)):
      n = len(globals()['predictions_val_%s' % sys][seed][~np.isnan(globals()['predictions_val_%s' % sys][seed])])
      m = len(globals()['y_valid_%s' % (seed+1)])
      if n >= m:
        ps = get_auprc(globals()['predictions_val_%s' % sys][seed][:m], globals()['y_valid_%s' % (seed+1)])
      else:
        ps = get_auprc(globals()['predictions_val_%s' % sys][seed][~np.isnan(globals()['predictions_val_%s' % sys][seed])], globals()['y_valid_%s' % (seed+1)][:n])
      ps_score[seed].append(ps)

def powerset(s):
    x = len(s)
    ls = []
    for i in range(1 << x):
        ls.append([s[j] for j in range(x) if (i & (1 << j))])
    return ls[1:]

models = powerset(scoreSys)

def myFunc(e):
  return len(e)

models.sort(key=myFunc)

models_list = []
for i in range(len(models)):
  if len(models[i]) == 1:
    models_list.append(models[i][0])
  elif len(models[i]) == 2:
    models_list.append(models[i][0]+'&'+models[i][1])
  elif len(models[i]) == 3:
    models_list.append(models[i][0]+'&'+models[i][1]+'&'+models[i][2])
  elif len(models[i]) == 4:
    models_list.append(models[i][0]+'&'+models[i][1]+'&'+models[i][2]+'&'+models[i][3])
  elif len(models[i]) == 5:
    models_list.append(models[i][0]+'&'+models[i][1]+'&'+models[i][2]+'&'+models[i][3]+'&'+models[i][4])
# Perform average score combinations
avg_score_combine_seed1 = pd.DataFrame({'xgb':predictions_test_xgb[0], 'rf':predictions_test_rf[0], 'svm':predictions_test_svm[0], 'adb':predictions_test_adb[0], 'cnn':predictions_test_cnn[0]})
avg_score_combine_seed2 = pd.DataFrame({'xgb':predictions_test_xgb[1], 'rf':predictions_test_rf[1], 'svm':predictions_test_svm[1], 'adb':predictions_test_adb[1], 'cnn':predictions_test_cnn[1]})
avg_score_combine_seed3 = pd.DataFrame({'xgb':predictions_test_xgb[2], 'rf':predictions_test_rf[2], 'svm':predictions_test_svm[2], 'adb':predictions_test_adb[2], 'cnn':predictions_test_cnn[2]})
avg_score_combine_seed4 = pd.DataFrame({'xgb':predictions_test_xgb[3], 'rf':predictions_test_rf[3], 'svm':predictions_test_svm[3], 'adb':predictions_test_adb[3], 'cnn':predictions_test_cnn[3]})
avg_score_combine_seed5 = pd.DataFrame({'xgb':predictions_test_xgb[4], 'rf':predictions_test_rf[4], 'svm':predictions_test_svm[4], 'adb':predictions_test_adb[4], 'cnn':predictions_test_cnn[4]})
def avg_score_combine(models_list, single_score):
  for j in models_list[len(scoreSys):]:
    if len(j.split('&')) == 2:
      single_score[j] = (single_score[j.split('&')[0]]+single_score[j.split('&')[1]]) / 2
    elif len(j.split('&')) == 3:
      single_score[j] = (single_score[j.split('&')[0]]+single_score[j.split('&')[1]]+single_score[j.split('&')[2]]) / 3
    elif len(j.split('&')) == 4:
      single_score[j] = (single_score[j.split('&')[0]]+single_score[j.split('&')[1]]+single_score[j.split('&')[2]]+single_score[j.split('&')[3]]) / 4
    elif len(j.split('&')) == 5:
      single_score[j] = (single_score[j.split('&')[0]]+single_score[j.split('&')[1]]+single_score[j.split('&')[2]]+single_score[j.split('&')[3]]+single_score[j.split('&')[4]]) / 5
avg_score_combine(models_list, avg_score_combine_seed1)
avg_score_combine(models_list, avg_score_combine_seed2)
avg_score_combine(models_list, avg_score_combine_seed3)
avg_score_combine(models_list, avg_score_combine_seed4)
avg_score_combine(models_list, avg_score_combine_seed5)
# Perform average rank combinations
avg_rank_combine_seed1 = pd.DataFrame({'xgb':score_to_rank(predictions_test_xgb[0]), 'rf':score_to_rank(predictions_test_rf[0]), 'svm':score_to_rank(predictions_test_svm[0]), 'adb':score_to_rank(predictions_test_adb[0]), 'cnn':score_to_rank(predictions_test_cnn[0])})
avg_rank_combine_seed2 = pd.DataFrame({'xgb':score_to_rank(predictions_test_xgb[1]), 'rf':score_to_rank(predictions_test_rf[1]), 'svm':score_to_rank(predictions_test_svm[1]), 'adb':score_to_rank(predictions_test_adb[1]), 'cnn':score_to_rank(predictions_test_cnn[1])})
avg_rank_combine_seed3 = pd.DataFrame({'xgb':score_to_rank(predictions_test_xgb[2]), 'rf':score_to_rank(predictions_test_rf[2]), 'svm':score_to_rank(predictions_test_svm[2]), 'adb':score_to_rank(predictions_test_adb[2]), 'cnn':score_to_rank(predictions_test_cnn[2])})
avg_rank_combine_seed4 = pd.DataFrame({'xgb':score_to_rank(predictions_test_xgb[3]), 'rf':score_to_rank(predictions_test_rf[3]), 'svm':score_to_rank(predictions_test_svm[3]), 'adb':score_to_rank(predictions_test_adb[3]), 'cnn':score_to_rank(predictions_test_cnn[3])})
avg_rank_combine_seed5 = pd.DataFrame({'xgb':score_to_rank(predictions_test_xgb[4]), 'rf':score_to_rank(predictions_test_rf[4]), 'svm':score_to_rank(predictions_test_svm[4]), 'adb':score_to_rank(predictions_test_adb[4]), 'cnn':score_to_rank(predictions_test_cnn[4])})
def avg_rank_combine(models_list, single_rank):
  for j in models_list[len(scoreSys):]:
    if len(j.split('&')) == 2:
      single_rank[j+'_r'] = (single_rank[j.split('&')[0]]+single_rank[j.split('&')[1]]) / 2
    elif len(j.split('&')) == 3:
      single_rank[j+'_r'] = (single_rank[j.split('&')[0]]+single_rank[j.split('&')[1]]+single_rank[j.split('&')[2]]) / 3
    elif len(j.split('&')) == 4:
      single_rank[j+'_r'] = (single_rank[j.split('&')[0]]+single_rank[j.split('&')[1]]+single_rank[j.split('&')[2]]+single_rank[j.split('&')[3]]) / 4
    elif len(j.split('&')) == 5:
      single_rank[j+'_r'] = (single_rank[j.split('&')[0]]+single_rank[j.split('&')[1]]+single_rank[j.split('&')[2]]+single_rank[j.split('&')[3]]+single_rank[j.split('&')[4]]) / 5
avg_rank_combine(models_list, avg_rank_combine_seed1)
avg_rank_combine(models_list, avg_rank_combine_seed2)
avg_rank_combine(models_list, avg_rank_combine_seed3)
avg_rank_combine(models_list, avg_rank_combine_seed4)
avg_rank_combine(models_list, avg_rank_combine_seed5)
# Perform weighted score combination by diversity strength
ds_score_combine_seed1 = pd.DataFrame()
ds_score_combine_seed2 = pd.DataFrame()
ds_score_combine_seed3 = pd.DataFrame()
ds_score_combine_seed4 = pd.DataFrame()
ds_score_combine_seed5 = pd.DataFrame()
def ds_score_combine(models_list, single_score, ds_score_combine, ds_score):
  for j in models_list[len(scoreSys):]:
    if len(j.split('&')) == 2:
      ds_score_combine[j+'_ds'] = (single_score[j.split('&')[0]]*ds_score[scoreSys.index(j.split('&')[0])]+single_score[j.split('&')[1]]*ds_score[scoreSys.index(j.split('&')[1])])/(ds_score[scoreSys.index(j.split('&')[0])] + ds_score[scoreSys.index(j.split('&')[1])])
    elif len(j.split('&')) == 3:
      ds_score_combine[j+'_ds'] = (single_score[j.split('&')[0]]*ds_score[scoreSys.index(j.split('&')[0])]+single_score[j.split('&')[1]]*ds_score[scoreSys.index(j.split('&')[1])]+single_score[j.split('&')[2]]*ds_score[scoreSys.index(j.split('&')[2])])/(ds_score[scoreSys.index(j.split('&')[0])] + ds_score[scoreSys.index(j.split('&')[1])] + ds_score[scoreSys.index(j.split('&')[2])])
    elif len(j.split('&')) == 4:
      ds_score_combine[j+'_ds'] = (single_score[j.split('&')[0]]*ds_score[scoreSys.index(j.split('&')[0])]+single_score[j.split('&')[1]]*ds_score[scoreSys.index(j.split('&')[1])]+single_score[j.split('&')[2]]*ds_score[scoreSys.index(j.split('&')[2])]+single_score[j.split('&')[3]]*ds_score[scoreSys.index(j.split('&')[3])])/(ds_score[scoreSys.index(j.split('&')[0])] + ds_score[scoreSys.index(j.split('&')[1])] + ds_score[scoreSys.index(j.split('&')[2])] + ds_score[scoreSys.index(j.split('&')[3])])
    elif len(j.split('&')) == 5:
      ds_score_combine[j+'_ds'] = (single_score[j.split('&')[0]]*ds_score[scoreSys.index(j.split('&')[0])]+single_score[j.split('&')[1]]*ds_score[scoreSys.index(j.split('&')[1])]+single_score[j.split('&')[2]]*ds_score[scoreSys.index(j.split('&')[2])]+single_score[j.split('&')[3]]*ds_score[scoreSys.index(j.split('&')[3])]+single_score[j.split('&')[4]]*ds_score[scoreSys.index(j.split('&')[4])])/(ds_score[scoreSys.index(j.split('&')[0])] + ds_score[scoreSys.index(j.split('&')[1])] + ds_score[scoreSys.index(j.split('&')[2])] + ds_score[scoreSys.index(j.split('&')[3])] + ds_score[scoreSys.index(j.split('&')[4])])
ds_score_combine(models_list, avg_score_combine_seed1, ds_score_combine_seed1, ds_score[0])
ds_score_combine(models_list, avg_score_combine_seed2, ds_score_combine_seed2, ds_score[1])
ds_score_combine(models_list, avg_score_combine_seed3, ds_score_combine_seed3, ds_score[2])
ds_score_combine(models_list, avg_score_combine_seed4, ds_score_combine_seed4, ds_score[3])
ds_score_combine(models_list, avg_score_combine_seed5, ds_score_combine_seed5, ds_score[4])
# Perform weighted rank combination by diversity strength
ds_rank_combine_seed1 = pd.DataFrame()
ds_rank_combine_seed2 = pd.DataFrame()
ds_rank_combine_seed3 = pd.DataFrame()
ds_rank_combine_seed4 = pd.DataFrame()
ds_rank_combine_seed5 = pd.DataFrame()
def ds_rank_combine(models_list, single_rank, ds_rank_combine, ds_rank):
  for j in models_list[len(scoreSys):]:
    if len(j.split('&')) == 2:
      ds_rank_combine[j+'_ds_r'] = (single_rank[j.split('&')[0]]*ds_rank[scoreSys.index(j.split('&')[0])]+single_rank[j.split('&')[1]]*ds_rank[scoreSys.index(j.split('&')[1])])/(ds_rank[scoreSys.index(j.split('&')[0])] + ds_rank[scoreSys.index(j.split('&')[1])])
    elif len(j.split('&')) == 3:
      ds_rank_combine[j+'_ds_r'] = (single_rank[j.split('&')[0]]*ds_rank[scoreSys.index(j.split('&')[0])]+single_rank[j.split('&')[1]]*ds_rank[scoreSys.index(j.split('&')[1])]+single_rank[j.split('&')[2]]*ds_rank[scoreSys.index(j.split('&')[2])])/(ds_rank[scoreSys.index(j.split('&')[0])] + ds_rank[scoreSys.index(j.split('&')[1])] + ds_rank[scoreSys.index(j.split('&')[2])])
    elif len(j.split('&')) == 4:
      ds_rank_combine[j+'_ds_r'] = (single_rank[j.split('&')[0]]*ds_rank[scoreSys.index(j.split('&')[0])]+single_rank[j.split('&')[1]]*ds_rank[scoreSys.index(j.split('&')[1])]+single_rank[j.split('&')[2]]*ds_rank[scoreSys.index(j.split('&')[2])]+single_rank[j.split('&')[3]]*ds_rank[scoreSys.index(j.split('&')[3])])/(ds_rank[scoreSys.index(j.split('&')[0])] + ds_rank[scoreSys.index(j.split('&')[1])] + ds_rank[scoreSys.index(j.split('&')[2])] + ds_rank[scoreSys.index(j.split('&')[3])])
    elif len(j.split('&')) == 5:
      ds_rank_combine[j+'_ds_r'] = (single_rank[j.split('&')[0]]*ds_rank[scoreSys.index(j.split('&')[0])]+single_rank[j.split('&')[1]]*ds_rank[scoreSys.index(j.split('&')[1])]+single_rank[j.split('&')[2]]*ds_rank[scoreSys.index(j.split('&')[2])]+single_rank[j.split('&')[3]]*ds_rank[scoreSys.index(j.split('&')[3])]+single_rank[j.split('&')[4]]*ds_rank[scoreSys.index(j.split('&')[4])])/(ds_rank[scoreSys.index(j.split('&')[0])] + ds_rank[scoreSys.index(j.split('&')[1])] + ds_rank[scoreSys.index(j.split('&')[2])] + ds_rank[scoreSys.index(j.split('&')[3])] + ds_rank[scoreSys.index(j.split('&')[4])])
ds_rank_combine(models_list, avg_rank_combine_seed1, ds_rank_combine_seed1, ds_rank[0])
ds_rank_combine(models_list, avg_rank_combine_seed2, ds_rank_combine_seed2, ds_rank[1])
ds_rank_combine(models_list, avg_rank_combine_seed3, ds_rank_combine_seed3, ds_rank[2])
ds_rank_combine(models_list, avg_rank_combine_seed4, ds_rank_combine_seed4, ds_rank[3])
ds_rank_combine(models_list, avg_rank_combine_seed5, ds_rank_combine_seed5, ds_rank[4])
# Perform weighted score combination by performance strength (AUROC/AUPRC/Spearman/MAE)
ps_score_combine_seed1 = pd.DataFrame()
ps_score_combine_seed2 = pd.DataFrame()
ps_score_combine_seed3 = pd.DataFrame()
ps_score_combine_seed4 = pd.DataFrame()
ps_score_combine_seed5 = pd.DataFrame()

if name in mae_datasets:
  ps_score = np.reciprocal(ps_score)

def ps_score_combine(models_list, single_score, ps_score_combine, ps_score):
  for j in models_list[len(scoreSys):]:
    if len(j.split('&')) == 2:
      ps_score_combine[j+'_ps'] = (single_score[j.split('&')[0]]*(ps_score[scoreSys.index(j.split('&')[0])])+single_score[j.split('&')[1]]*(ps_score[scoreSys.index(j.split('&')[1])]))/(ps_score[scoreSys.index(j.split('&')[0])] + ps_score[scoreSys.index(j.split('&')[1])])
    elif len(j.split('&')) == 3:
      ps_score_combine[j+'_ps'] = (single_score[j.split('&')[0]]*(ps_score[scoreSys.index(j.split('&')[0])])+single_score[j.split('&')[1]]*(ps_score[scoreSys.index(j.split('&')[1])])+single_score[j.split('&')[2]]*(ps_score[scoreSys.index(j.split('&')[2])]))/(ps_score[scoreSys.index(j.split('&')[0])] + ps_score[scoreSys.index(j.split('&')[1])] + ps_score[scoreSys.index(j.split('&')[2])])
    elif len(j.split('&')) == 4:
      ps_score_combine[j+'_ps'] = (single_score[j.split('&')[0]]*(ps_score[scoreSys.index(j.split('&')[0])])+single_score[j.split('&')[1]]*(ps_score[scoreSys.index(j.split('&')[1])])+single_score[j.split('&')[2]]*(ps_score[scoreSys.index(j.split('&')[2])])+single_score[j.split('&')[3]]*(ps_score[scoreSys.index(j.split('&')[3])]))/(ps_score[scoreSys.index(j.split('&')[0])] + ps_score[scoreSys.index(j.split('&')[1])] + ps_score[scoreSys.index(j.split('&')[2])] + ps_score[scoreSys.index(j.split('&')[3])])
    elif len(j.split('&')) == 5:
      ps_score_combine[j+'_ps'] = (single_score[j.split('&')[0]]*(ps_score[scoreSys.index(j.split('&')[0])])+single_score[j.split('&')[1]]*(ps_score[scoreSys.index(j.split('&')[1])])+single_score[j.split('&')[2]]*(ps_score[scoreSys.index(j.split('&')[2])])+single_score[j.split('&')[3]]*(ps_score[scoreSys.index(j.split('&')[3])])+single_score[j.split('&')[4]]*(ps_score[scoreSys.index(j.split('&')[4])]))/(ps_score[scoreSys.index(j.split('&')[0])] + ps_score[scoreSys.index(j.split('&')[1])] + ps_score[scoreSys.index(j.split('&')[2])] + ps_score[scoreSys.index(j.split('&')[3])] + ps_score[scoreSys.index(j.split('&')[4])])
ps_score_combine(models_list, avg_score_combine_seed1, ps_score_combine_seed1, ps_score[0])
ps_score_combine(models_list, avg_score_combine_seed2, ps_score_combine_seed2, ps_score[1])
ps_score_combine(models_list, avg_score_combine_seed3, ps_score_combine_seed3, ps_score[2])
ps_score_combine(models_list, avg_score_combine_seed4, ps_score_combine_seed4, ps_score[3])
ps_score_combine(models_list, avg_score_combine_seed5, ps_score_combine_seed5, ps_score[4])
# Perform weighted rank combination by performance strength
ps_rank_combine_seed1 = pd.DataFrame()
ps_rank_combine_seed2 = pd.DataFrame()
ps_rank_combine_seed3 = pd.DataFrame()
ps_rank_combine_seed4 = pd.DataFrame()
ps_rank_combine_seed5 = pd.DataFrame()
def ps_rank_combine(models_list, single_rank, ps_rank_combine, ps_score):
  for j in models_list[len(scoreSys):]:
    if len(j.split('&')) == 2:
      ps_rank_combine[j+'_ps_r'] = (single_rank[j.split('&')[0]]*(1 / ps_score[scoreSys.index(j.split('&')[0])])+single_rank[j.split('&')[1]]*(1 / ps_score[scoreSys.index(j.split('&')[1])]))/(1 / ps_score[scoreSys.index(j.split('&')[0])] + 1 / ps_score[scoreSys.index(j.split('&')[1])])
    elif len(j.split('&')) == 3:
      ps_rank_combine[j+'_ps_r'] = (single_rank[j.split('&')[0]]*(1 / ps_score[scoreSys.index(j.split('&')[0])])+single_rank[j.split('&')[1]]*(1 / ps_score[scoreSys.index(j.split('&')[1])])+single_rank[j.split('&')[2]]*(1 / ps_score[scoreSys.index(j.split('&')[2])]))/(1 / ps_score[scoreSys.index(j.split('&')[0])] + 1 / ps_score[scoreSys.index(j.split('&')[1])] + 1 / ps_score[scoreSys.index(j.split('&')[2])])
    elif len(j.split('&')) == 4:
      ps_rank_combine[j+'_ps_r'] = (single_rank[j.split('&')[0]]*(1 / ps_score[scoreSys.index(j.split('&')[0])])+single_rank[j.split('&')[1]]*(1 / ps_score[scoreSys.index(j.split('&')[1])])+single_rank[j.split('&')[2]]*(1 / ps_score[scoreSys.index(j.split('&')[2])])+single_rank[j.split('&')[3]]*(1 / ps_score[scoreSys.index(j.split('&')[3])]))/(1 / ps_score[scoreSys.index(j.split('&')[0])] + 1 / ps_score[scoreSys.index(j.split('&')[1])] + 1 / ps_score[scoreSys.index(j.split('&')[2])] + 1 / ps_score[scoreSys.index(j.split('&')[3])])
    elif len(j.split('&')) == 5:
      ps_rank_combine[j+'_ps_r'] = (single_rank[j.split('&')[0]]*(1 / ps_score[scoreSys.index(j.split('&')[0])])+single_rank[j.split('&')[1]]*(1 / ps_score[scoreSys.index(j.split('&')[1])])+single_rank[j.split('&')[2]]*(1 / ps_score[scoreSys.index(j.split('&')[2])])+single_rank[j.split('&')[3]]*(1 / ps_score[scoreSys.index(j.split('&')[3])])+single_rank[j.split('&')[4]]*(1 / ps_score[scoreSys.index(j.split('&')[4])]))/(1 / ps_score[scoreSys.index(j.split('&')[0])] + 1 / ps_score[scoreSys.index(j.split('&')[1])] + 1 / ps_score[scoreSys.index(j.split('&')[2])] + 1 / ps_score[scoreSys.index(j.split('&')[3])] + 1 / ps_score[scoreSys.index(j.split('&')[4])])
ps_rank_combine(models_list, avg_rank_combine_seed1, ps_rank_combine_seed1, ps_score[0])
ps_rank_combine(models_list, avg_rank_combine_seed2, ps_rank_combine_seed2, ps_score[1])
ps_rank_combine(models_list, avg_rank_combine_seed3, ps_rank_combine_seed3, ps_score[2])
ps_rank_combine(models_list, avg_rank_combine_seed4, ps_rank_combine_seed4, ps_score[3])
ps_rank_combine(models_list, avg_rank_combine_seed5, ps_rank_combine_seed5, ps_score[4])
avg_rank_combine_seed1.rename(columns={'xgb': 'xgb_r', 'rf': 'rf_r', 'svm': 'svm_r', 'adb': 'adb_r', 'cnn': 'cnn_r'}, inplace=True)
avg_rank_combine_seed2.rename(columns={'xgb': 'xgb_r', 'rf': 'rf_r', 'svm': 'svm_r', 'adb': 'adb_r', 'cnn': 'cnn_r'}, inplace=True)
avg_rank_combine_seed3.rename(columns={'xgb': 'xgb_r', 'rf': 'rf_r', 'svm': 'svm_r', 'adb': 'adb_r', 'cnn': 'cnn_r'}, inplace=True)
avg_rank_combine_seed4.rename(columns={'xgb': 'xgb_r', 'rf': 'rf_r', 'svm': 'svm_r', 'adb': 'adb_r', 'cnn': 'cnn_r'}, inplace=True)
avg_rank_combine_seed5.rename(columns={'xgb': 'xgb_r', 'rf': 'rf_r', 'svm': 'svm_r', 'adb': 'adb_r', 'cnn': 'cnn_r'}, inplace=True)

# get MAE/Spearman/AUROC/AUPRC
score_combine_list = np.hstack((np.array(avg_score_combine_seed1.columns), np.array(ds_score_combine_seed1.columns), np.array(ps_score_combine_seed1.columns)))
rank_combine_list = np.hstack((np.array(avg_rank_combine_seed1.columns), np.array(ds_rank_combine_seed1.columns), np.array(ps_rank_combine_seed1.columns)))
pd.set_option("display.max_rows", None)

if name in mae_datasets:
  MAE = pd.DataFrame(index = score_combine_list)
  for i in range(1, 6):
    mae_avg, mae_ds, mae_ps = [], [], []
    for col in globals()['avg_score_combine_seed%s' %i].columns:
      mae = get_mae(np.array(globals()['avg_score_combine_seed%s' %i][col]), y_test)
      mae_avg.append(mae)
    for col in globals()['ds_score_combine_seed%s' %i].columns:
      mae = get_mae(np.array(globals()['ds_score_combine_seed%s' %i][col]), y_test)
      mae_ds.append(mae)
    for col in globals()['ps_score_combine_seed%s' %i].columns:
      mae = get_mae(np.array(globals()['ps_score_combine_seed%s' %i][col]), y_test)
      mae_ps.append(mae)
    MAE['seed'+str(i)] = np.hstack((mae_avg, mae_ds, mae_ps))
  MAE['avg_MAE'] = MAE.mean(axis=1)
  print(MAE.sort_values(by='avg_MAE'))
elif name in spearman_datasets:
  spearman_corr = pd.DataFrame(index = rank_combine_list)
  for i in range(1, 6):
    sc_avg, sc_ds, sc_ps = [], [], []
    for col in globals()['avg_rank_combine_seed%s' %i].columns:
      sc = get_spearman_corr(np.array(globals()['avg_rank_combine_seed%s' %i][col]), score_to_rank(y_test))
      sc_avg.append(sc)
    for col in globals()['ds_rank_combine_seed%s' %i].columns:
      sc = get_spearman_corr(np.array(globals()['ds_rank_combine_seed%s' %i][col]), score_to_rank(y_test))
      sc_ds.append(sc)
    for col in globals()['ps_rank_combine_seed%s' %i].columns:
      sc = get_spearman_corr(np.array(globals()['ps_rank_combine_seed%s' %i][col]), score_to_rank(y_test))
      sc_ps.append(sc)
    spearman_corr['seed'+str(i)] = np.hstack((sc_avg, sc_ds, sc_ps))
  spearman_corr['avg_spearman'] = spearman_corr.mean(axis=1)
  print(spearman_corr.sort_values(by='avg_spearman', ascending=False))
elif name in auroc_datasets:
  AUROC = pd.DataFrame(index = score_combine_list)
  for i in range(1, 6):
    auroc_avg, auroc_ds, auroc_ps = [], [], []
    for col in globals()['avg_score_combine_seed%s' %i].columns:
      auroc_score = get_auroc(np.array(globals()['avg_score_combine_seed%s' %i][col]), y_test)
      auroc_avg.append(auroc_score)
    for col in globals()['ds_score_combine_seed%s' %i].columns:
      auroc_score = get_auroc(np.array(globals()['ds_score_combine_seed%s' %i][col]), y_test)
      auroc_ds.append(auroc_score)
    for col in globals()['ps_score_combine_seed%s' %i].columns:
      auroc_score = get_auroc(np.array(globals()['ps_score_combine_seed%s' %i][col]), y_test)
      auroc_ps.append(auroc_score)
    AUROC['seed'+str(i)] = np.hstack((auroc_avg, auroc_ds, auroc_ps))
  AUROC['avg_AUROC'] = AUROC.mean(axis=1)
  print(AUROC.sort_values(by='avg_AUROC', ascending=False))
elif name in auprc_datasets:
  AUPRC = pd.DataFrame(index = score_combine_list)
  for i in range(1, 6):
    auprc_avg, auprc_ds, auprc_ps = [], [], []
    for col in globals()['avg_score_combine_seed%s' %i].columns:
      auprc_score = get_auprc(np.array(globals()['avg_score_combine_seed%s' %i][col]), y_test)
      auprc_avg.append(auprc_score)
    for col in globals()['ds_score_combine_seed%s' %i].columns:
      auprc_score = get_auprc(np.array(globals()['ds_score_combine_seed%s' %i][col]), y_test)
      auprc_ds.append(auprc_score)
    for col in globals()['ps_score_combine_seed%s' %i].columns:
      auprc_score = get_auprc(np.array(globals()['ps_score_combine_seed%s' %i][col]), y_test)
      auprc_ps.append(auprc_score)
    AUPRC['seed'+str(i)] = np.hstack((auprc_avg, auprc_ds, auprc_ps))
  AUPRC['avg_AUPRC'] = AUPRC.mean(axis=1)
  print(AUPRC.sort_values(by='avg_AUPRC', ascending=False))



Found local copy...
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 674.07it/s]
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 616.25it/s]
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 709.62it/s]
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 417.26it/s]
generating training, validation splits...
100%|██████████| 379/379 [00:00<00:00, 550.56it/s]


                          seed1     seed2     seed3     seed4     seed5  \
xgb&rf&svm_ps          0.922174  0.927826  0.899565  0.911739  0.936087   
xgb&rf&svm             0.921739  0.928261  0.897391  0.906522  0.936522   
xgb&rf&svm_ds          0.926087  0.919565  0.894348  0.911304  0.938261   
xgb&rf&svm&adb         0.931304  0.911739  0.893043  0.913913  0.933913   
xgb&rf&svm&adb_ps      0.930435  0.911739  0.893913  0.912609  0.933478   
xgb&svm&adb            0.924783  0.904783  0.891739  0.911304  0.933478   
xgb&svm&adb_ps         0.924783  0.903913  0.891304  0.909130  0.931739   
xgb&rf&svm&adb_ds      0.926957  0.907391  0.890870  0.905652  0.929565   
rf&svm_ds              0.924783  0.910870  0.900870  0.901304  0.917391   
rf&svm&adb_ps          0.932174  0.898696  0.890000  0.904348  0.920870   
rf&svm&adb             0.933043  0.898696  0.889565  0.904783  0.919130   
xgb&svm&adb_ds         0.922609  0.903913  0.890435  0.899130  0.924348   
xgb&svm_ds             0.

In [ ]:
np.array(ds_score).mean(axis=0)

array([0.65954787, 0.56610223, 0.45451577, 0.79404678, 2.15637621])